In [38]:
import pandas as pd
import pyxlsb
from datetime import date, timedelta
import numpy as np

In [39]:
estoque_dtype = {
    'Código Produto':int,
    'Código Estabelecimento':int,
    'Total': 'int64',
    'Data':object
}


estoque = pd.read_excel("Estoque_Por_Dia.xlsx",
usecols=['Código Produto','Código Estabelecimento','Total','Data'],
parse_dates=['Data'],
dtype=estoque_dtype)


estoque.rename(columns={'Total':'Estoque'},inplace=True)



In [40]:
estoque['Data'] = pd.to_datetime(estoque['Data'],dayfirst=True)
estoque.head(10)

,Código Produto,Código Estabelecimento,Estoque,Data
0,4110020335,804,1,2025-06-09
1,4110020335,806,1,2025-06-09
2,4110020336,805,1,2025-06-09
3,4110020336,806,1,2025-06-09
4,4110020337,804,1,2025-06-09
5,4110020337,806,2,2025-06-09
6,4110020337,807,1,2025-06-09
7,4110020488,805,1,2025-06-09
8,4110020498,804,1,2025-06-09
9,4110023035,806,1,2025-06-09


In [41]:
Vendas_dtype = {
    'Código Produto':int,
    'Código Estabelecimento':int,
    'Quantidade Faturada': 'int64',
    'Fat Bruto':float,
    'Dia do Mês':object
}



Vendas = pd.read_excel('Cubo de Vendas.xlsx',skiprows=9,
usecols=['Código Produto','Código Estabelecimento','Quantidade Faturada','Fat Bruto','Dia do Mês'],
parse_dates=['Dia do Mês'],
dtype=Vendas_dtype)

Vendas.rename(columns={'Dia do Mês':'Data'},inplace=True)

Vendas['Data'] = pd.to_datetime(Vendas['Data'],dayfirst=True)

In [42]:
#Vendas.head(10)

In [43]:

data_hoje = date.today()
dias_atras = 90
data_inicio = data_hoje - timedelta(days=dias_atras - 1)
df_datas = pd.DataFrame(
    pd.date_range(start=data_inicio, end=data_hoje)
)
df_datas.rename(columns={0: 'Data'}, inplace=True)

print(df_datas.head(1))
print(df_datas.tail(1))

        Data
0 2025-06-10
         Data
89 2025-09-07


In [44]:
df_produtos_lojas = pd.concat([estoque[['Código Produto', 'Código Estabelecimento']], Vendas[['Código Produto', 'Código Estabelecimento']]]).drop_duplicates()

df_base_completa = pd.merge(df_datas, df_produtos_lojas, how='cross')


df_final = pd.merge(df_base_completa, estoque, on=['Data', 'Código Produto', 'Código Estabelecimento'], how='left')

df_final = pd.merge(df_final, Vendas, on=['Data', 'Código Produto', 'Código Estabelecimento'], how='left')

df_final = df_final.fillna(0)

df_final.sort_values(by=['Código Estabelecimento', 'Código Produto', 'Data'], inplace=True)
df_final.reset_index(drop=True, inplace=True)


In [45]:
df_final.head(10)
##df_final.to_excel('teste.xlsx')

Qtd_Faturada = df_final['Quantidade Faturada'].sum()
Fat_Bruto = df_final['Fat Bruto'].sum()

print('qtd faturada = ',f"{Qtd_Faturada:,.2f}", ' Fat bruto = ',f"{Fat_Bruto:,.2f}")

qtd faturada =  91,500.00  Fat bruto =  29,567,144.17


In [46]:
df_rupturas = df_final[(df_final['Quantidade Faturada'] > 0) & (df_final['Estoque'] == 0)]

df_rupturas.head(10)

,Data,Código Produto,Código Estabelecimento,Estoque,Quantidade Faturada,Fat Bruto
325,2025-08-04,4110853666,800,0.0,1.0,41.99
482,2025-07-12,4111110281,800,0.0,1.0,449.99
582,2025-07-22,4111110294,800,0.0,1.0,6299.99
608,2025-08-17,4111110294,800,0.0,1.0,5949.99
901,2025-06-11,4111110311,800,0.0,1.0,499.99
902,2025-06-12,4111110311,800,0.0,3.0,1499.97
919,2025-06-29,4111110311,800,0.0,1.0,499.99
975,2025-08-24,4111110311,800,0.0,1.0,499.99
1022,2025-07-12,4111110313,800,0.0,1.0,559.99
1128,2025-07-28,4111110314,800,0.0,1.0,479.99


In [47]:

df_final_ajustado = df_final.copy()

df_final_ajustado.loc[df_final_ajustado['Estoque'] < 0, 'Estoque'] = 0

condicao = (df_final_ajustado['Quantidade Faturada'] > 0) & (df_final_ajustado['Estoque'] == 0)

df_final_ajustado.loc[condicao, 'Estoque'] = df_final_ajustado.loc[condicao, 'Quantidade Faturada']


Qtd_Faturada = df_final_ajustado['Quantidade Faturada'].sum()
Fat_Bruto = df_final_ajustado['Fat Bruto'].sum()

print('qtd faturada = ',f"{Qtd_Faturada:,.2f}", ' Fat bruto = ',f"{Fat_Bruto:,.2f}")

qtd faturada =  91,500.00  Fat bruto =  29,567,144.17


In [48]:
def realizar_analise(df, group_cols):
    qnt_faturada_col = 'Quantidade_Faturada' if 'Quantidade_Faturada' in df.columns else 'Quantidade Faturada'
    fat_bruto_col = 'Fat_Bruto' if 'Fat_Bruto' in df.columns else 'Fat Bruto'

    df_analise = df[df['Estoque'] > 0].groupby(group_cols).agg(
        dias_com_estoque=('Data', 'count'),
        estoque_medio=('Estoque', 'mean'),
        venda_media=(qnt_faturada_col, 'mean'),
        desvio_padrao=(qnt_faturada_col, 'std'),
        quantidade_faturada_total=(qnt_faturada_col, 'sum')
    ).reset_index()

    df_abc = df.groupby(group_cols)[fat_bruto_col].sum().reset_index()

    if 'Código Estabelecimento' in group_cols:
        # Lógica corrigida para calcular o ABC por loja
        df_abc['rank_loja'] = df_abc.groupby('Código Estabelecimento')[fat_bruto_col].rank(ascending=False)
        df_abc.sort_values(by=['Código Estabelecimento', 'rank_loja'], inplace=True)
        df_abc['fat_acumulado'] = df_abc.groupby('Código Estabelecimento')[fat_bruto_col].cumsum()
        
        # CORREÇÃO: Usando .transform() para evitar erro de índice
        df_abc['%_acumulado'] = df_abc['fat_acumulado'] / df_abc.groupby('Código Estabelecimento')['fat_acumulado'].transform('max')
    else:
        df_abc.sort_values(by=fat_bruto_col, ascending=False, inplace=True)
        df_abc['fat_acumulado'] = df_abc[fat_bruto_col].cumsum()
        df_abc['%_acumulado'] = df_abc['fat_acumulado'] / df_abc[fat_bruto_col].sum()
    
    def classificar_abc(porcentagem):
        if porcentagem <= 0.70:
            return 'A'
        elif porcentagem <= 0.95:
            return 'B'
        else:
            return 'C'
    
    df_abc['Classificacao_ABC'] = df_abc['%_acumulado'].apply(classificar_abc)

    df_resultados = pd.merge(df_analise, df_abc.drop(columns=['fat_acumulado', '%_acumulado']), on=group_cols, how='left')
    
    if 'rank_loja' in df_resultados.columns:
        df_resultados.drop(columns=['rank_loja'], inplace=True)

    df_resultados['CV'] = np.where(df_resultados['venda_media'] > 0, 
                                 df_resultados['desvio_padrao'] / df_resultados['venda_media'], 
                                 np.nan)

    return df_resultados

In [49]:
analise_por_loja = realizar_analise(df_final_ajustado, ['Código Estabelecimento', 'Código Produto'])

df_geral_consolidado = df_final_ajustado.groupby(['Data', 'Código Produto']).agg(
    Estoque=('Estoque', 'sum'),
    Quantidade_Faturada=('Quantidade Faturada', 'sum'),
    Fat_Bruto=('Fat Bruto', 'sum')
).reset_index()

analise_geral = realizar_analise(df_geral_consolidado, ['Código Produto'])

In [50]:
with pd.ExcelWriter('analise_completa.xlsx', engine='xlsxwriter') as writer:
    analise_por_loja.to_excel(writer, sheet_name='Análise por Loja', index=False)
    analise_geral.to_excel(writer, sheet_name='Análise Geral', index=False)

print("Análises completas geradas com sucesso no arquivo 'analise_completa.xlsx'!")

Análises completas geradas com sucesso no arquivo 'analise_completa.xlsx'!


In [51]:
def realizar_analise_semanal(df, group_cols):
    qnt_faturada_col = 'Quantidade_Faturada' if 'Quantidade_Faturada' in df.columns else 'Quantidade Faturada'
    fat_bruto_col = 'Fat_Bruto' if 'Fat_Bruto' in df.columns else 'Fat Bruto'

    # 1. Contagem de dias com estoque > 0 e estoque médio (diário)
    df_analise = df[df['Estoque'] > 0].groupby(group_cols).agg(
        dias_com_estoque=('Data', 'count'),
        estoque_medio=('Estoque', 'mean'),
    ).reset_index()

    # 2. Consolidação semanal das vendas para cálculo de desvio padrão e CV
    df_vendas = df[df['Estoque'] > 0].copy()
    df_vendas['semana_ano'] = df_vendas['Data'].dt.isocalendar().week.astype(str) + '-' + df_vendas['Data'].dt.isocalendar().year.astype(str)
    
    group_semanal = group_cols + ['semana_ano']
    df_vendas_semanal = df_vendas.groupby(group_semanal).agg(
        venda_semanal_total=(qnt_faturada_col, 'sum')
    ).reset_index()

    df_vendas_analise_semanal = df_vendas_semanal.groupby(group_cols).agg(
        venda_media_semanal=('venda_semanal_total', 'mean'),
        desvio_padrao_semanal=('venda_semanal_total', 'std')
    ).reset_index()
    
    # 3. Classificação ABC (baseado no faturamento total)
    df_abc = df.groupby(group_cols)[fat_bruto_col].sum().reset_index()

    if 'Código Estabelecimento' in group_cols:
        df_abc['rank_loja'] = df_abc.groupby('Código Estabelecimento')[fat_bruto_col].rank(ascending=False)
        df_abc.sort_values(by=['Código Estabelecimento', 'rank_loja'], inplace=True)
        df_abc['fat_acumulado'] = df_abc.groupby('Código Estabelecimento')[fat_bruto_col].cumsum()
        df_abc['%_acumulado'] = df_abc['fat_acumulado'] / df_abc.groupby('Código Estabelecimento')[fat_bruto_col].transform('sum')
    else:
        df_abc.sort_values(by=fat_bruto_col, ascending=False, inplace=True)
        df_abc['fat_acumulado'] = df_abc[fat_bruto_col].cumsum()
        df_abc['%_acumulado'] = df_abc['fat_acumulado'] / df_abc[fat_bruto_col].sum()
    
    def classificar_abc(porcentagem):
        if porcentagem <= 0.70:
            return 'A'
        elif porcentagem <= 0.95:
            return 'B'
        else:
            return 'C'
    
    df_abc['Classificacao_ABC'] = df_abc['%_acumulado'].apply(classificar_abc)

    # 4. Junção dos resultados e cálculo do CV
    df_resultados = pd.merge(df_analise, df_vendas_analise_semanal, on=group_cols, how='left')
    df_resultados = pd.merge(df_resultados, df_abc.drop(columns=['fat_acumulado', '%_acumulado']), on=group_cols, how='left')
    
    if 'rank_loja' in df_resultados.columns:
        df_resultados.drop(columns=['rank_loja'], inplace=True)

    df_resultados['CV'] = np.where(df_resultados['venda_media_semanal'] > 0, 
                                 df_resultados['desvio_padrao_semanal'] / df_resultados['venda_media_semanal'], 
                                 np.nan)
    
    return df_resultados

In [52]:
df_geral_consolidado_semanal = df_final_ajustado.groupby(['Data', 'Código Produto']).agg(
    Estoque=('Estoque', 'sum'),
    **{'Quantidade Faturada':('Quantidade Faturada', 'sum')},
    **{'Fat Bruto':('Fat Bruto', 'sum')}
).reset_index()
analise_semanal = realizar_analise_semanal(df_geral_consolidado_semanal, ['Código Produto'])

analise_semanal_por_loja = realizar_analise_semanal(df_final_ajustado, ['Código Estabelecimento', 'Código Produto'])    


with pd.ExcelWriter('analise_completa semanal.xlsx', engine='xlsxwriter') as writer:
    analise_por_loja.to_excel(writer, sheet_name='Análise por Loja', index=False)
    analise_geral.to_excel(writer, sheet_name='Análise Geral', index=False)
    analise_semanal.to_excel(writer, sheet_name='Análise Semanal', index=False)
    analise_semanal_por_loja.to_excel(writer,sheet_name='Análise Semanal Por Loja', index=False)


print("Análises completas geradas com sucesso no arquivo 'analise_completa.xlsx'!")


Análises completas geradas com sucesso no arquivo 'analise_completa.xlsx'!


In [ ]:

def realizar_analise_mensal(df, group_cols):
    qnt_faturada_col = 'Quantidade_Faturada' if 'Quantidade_Faturada' in df.columns else 'Quantidade Faturada'
    fat_bruto_col = 'Fat_Bruto' if 'Fat_Bruto' in df.columns else 'Fat Bruto'
    df_analise = df[df['Estoque'] > 0].groupby(group_cols).agg(
        dias_com_estoque=('Data', 'count'),
        estoque_medio=('Estoque', 'mean'),
    ).reset_index()
    df_vendas = df[df['Estoque'] > 0].copy()
    
   
    df_vendas['janela_30_dias'] = df_vendas['Data'].apply(lambda x: (x - df_vendas['Data'].min()).days // 30)
    
    group_mensal = group_cols + ['janela_30_dias']
    df_vendas_mensal = df_vendas.groupby(group_mensal).agg(
        venda_mensal_total=(qnt_faturada_col, 'sum')
    ).reset_index()
    
    df_vendas_analise_mensal = df_vendas_mensal.groupby(group_cols).agg(
        venda_media_mensal=('venda_mensal_total', 'mean'),
        desvio_padrao_mensal=('venda_mensal_total', 'std')
    ).reset_index()
    
    df_abc = df.groupby(group_cols)[fat_bruto_col].sum().reset_index()
    if 'Código Estabelecimento' in group_cols:
        df_abc['rank_loja'] = df_abc.groupby('Código Estabelecimento')[fat_bruto_col].rank(ascending=False)
        df_abc.sort_values(by=['Código Estabelecimento', 'rank_loja'], inplace=True)
        df_abc['fat_acumulado'] = df_abc.groupby('Código Estabelecimento')[fat_bruto_col].cumsum()
        df_abc['%_acumulado'] = df_abc['fat_acumulado'] / df_abc.groupby('Código Estabelecimento')[fat_bruto_col].transform('sum')
    else:
        df_abc.sort_values(by=fat_bruto_col, ascending=False, inplace=True)
        df_abc['fat_acumulado'] = df_abc[fat_bruto_col].cumsum()
        df_abc['%_acumulado'] = df_abc['fat_acumulado'] / df_abc[fat_bruto_col].sum()
    
    def classificar_abc(porcentagem):
        if porcentagem <= 0.70: return 'A'
        elif porcentagem <= 0.95: return 'B'
        else: return 'C'
    
    df_abc['Classificacao_ABC'] = df_abc['%_acumulado'].apply(classificar_abc)
    df_resultados = pd.merge(df_analise, df_vendas_analise_mensal, on=group_cols, how='left')
    df_resultados = pd.merge(df_resultados, df_abc.drop(columns=['fat_acumulado', '%_acumulado']), on=group_cols, how='left')
    
    if 'rank_loja' in df_resultados.columns:
        df_resultados.drop(columns=['rank_loja'], inplace=True)
    
    df_resultados['CV'] = np.where(df_resultados['venda_media_mensal'] > 0,
                                 df_resultados['desvio_padrao_mensal'] / df_resultados['venda_media_mensal'],
                                 np.nan)
    return df_resultados


analise_mensal_por_loja = realizar_analise_mensal(df_final_ajustado, ['Código Estabelecimento', 'Código Produto'])
analise_mensal_geral = realizar_analise_mensal(df_geral_consolidado, ['Código Produto'])


with pd.ExcelWriter('analise_completa.xlsx', engine='xlsxwriter') as writer:
    analise_por_loja.to_excel(writer, sheet_name='Análise por Loja', index=False)
    analise_geral.to_excel(writer, sheet_name='Análise Geral', index=False)
    analise_semanal.to_excel(writer, sheet_name='Análise Semanal', index=False)
    analise_semanal_por_loja.to_excel(writer,sheet_name='Análise Semanal Por Loja', index=False)
    analise_mensal_por_loja.to_excel(writer, sheet_name='Análise Mensal por Loja', index=False)
    analise_mensal_geral.to_excel(writer, sheet_name='Análise Mensal Geral', index=False)
    
print("Análises completas geradas com sucesso no arquivo 'analise_completa.xlsx'!")